In [1]:
import pandas as pd
from ast import literal_eval
import seaborn as sns
import numpy as np


movies_metadata = pd.read_csv("data/movies_metadata.csv")
links_small  = pd.read_csv("data/links_small.csv")
movies_keywords  = pd.read_csv("data/keywords.csv")

links_small = links_small[links_small['tmdbId'].notnull()]["tmdbId"].astype('int')
movies_metadata_small = movies_metadata[movies_metadata['id'].isin(links_small.astype('str'))]

movies_metadata_small.shape




C:\Users\Admin\AppData\Local\Temp\ipykernel_2084\2495644953.py:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv("data/movies_metadata.csv")


(9099, 24)

In [2]:
movies_metadata_small['id'] # object
movies_keywords['id'] = movies_keywords['id'].astype('str') # int -> object


In [3]:
movies_metadata_small = movies_metadata_small.merge(movies_keywords, on='id')

In [4]:
movies_metadata_small.head() # keywords

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [5]:
movies = movies_metadata_small[['title', 'genres','keywords','overview','popularity', 'release_date']]

In [6]:
movies.head()

,title,genres,keywords,overview,popularity,release_date
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",When siblings Judy and Peter discover an encha...,17.015539,1995-12-15
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",A family wedding reignites the ancient feud be...,11.7129,1995-12-22
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'id': 818, 'name': 'based on novel'}, {'id':...","Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",Just when George Banks has recovered from his ...,8.387519,1995-02-10


In [7]:
movies['str_genres'] = movies['genres'].fillna('[]') \
                    .apply(literal_eval) \
                    .apply(lambda x: sorted(i['name'] for i in x ) if isinstance(x,list) else []) \
                    .apply(lambda x: ", ".join(x) if len(x) > 0 else None)



C:\Users\Admin\AppData\Local\Temp\ipykernel_2084\1845854695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['str_genres'] = movies['genres'].fillna('[]') \


In [8]:
movies.head()

,title,genres,keywords,overview,popularity,release_date,str_genres
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,"Animation, Comedy, Family"
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,"Adventure, Family, Fantasy"
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",A family wedding reignites the ancient feud be...,11.7129,1995-12-22,"Comedy, Romance"
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'id': 818, 'name': 'based on novel'}, {'id':...","Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,"Comedy, Drama, Romance"
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",Just when George Banks has recovered from his ...,8.387519,1995-02-10,Comedy


In [9]:
movies['str_keywords'] = movies['keywords'].fillna('[]') \
                    .apply(literal_eval) \
                    .apply(lambda x: sorted(i['name'] for i in x ) if isinstance(x,list) else []) \
                    .apply(lambda x: ", ".join(x) if len(x) > 0 else None)

C:\Users\Admin\AppData\Local\Temp\ipykernel_2084\4060563036.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['str_keywords'] = movies['keywords'].fillna('[]') \


In [10]:
movies['str_keywords'].head()

0    boy, boy next door, friends, friendship, jealo...
1    based on children's book, board game, disappea...
2    best friend, duringcreditsstinger, fishing, ol...
3    based on novel, chick flick, divorce, interrac...
4    aging, baby, confidence, contraception, daught...
Name: str_keywords, dtype: object

In [11]:
def get_genres_keywords_overview(row):
    return f"""
    genres:
    {row['str_genres']}

    
    keywords:
    {row['str_keywords']}

    overview:
    {row['overview']}

    """


In [12]:
movies['genres_keywords_overview'] = movies.apply(get_genres_keywords_overview,axis=1)

C:\Users\Admin\AppData\Local\Temp\ipykernel_2084\2575982863.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['genres_keywords_overview'] = movies.apply(get_genres_keywords_overview,axis=1)


In [13]:
movies['genres_keywords_overview'].iloc[0]

"\n    genres:\n    Animation, Comedy, Family\n\n\n    keywords:\n    boy, boy next door, friends, friendship, jealousy, new toy, rivalry, toy, toy comes to life\n\n    overview:\n    Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.\n\n    "

In [14]:
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')
movies['year'] = movies['release_date'].dt.year

movies['popularity'] = movies['popularity'].astype(float)
movies['popularity_log'] = np.log(movies['popularity'])

C:\Users\Admin\AppData\Local\Temp\ipykernel_2084\2474586242.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')
C:\Users\Admin\AppData\Local\Temp\ipykernel_2084\2474586242.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['year'] = movies['release_date'].dt.year
C:\Users\Admin\AppData\Local\Temp\ipykernel_2084\2474586242.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [15]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9151 entries, 0 to 9150
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   title                     9151 non-null   object        
 1   genres                    9151 non-null   object        
 2   keywords                  9151 non-null   object        
 3   overview                  9139 non-null   object        
 4   popularity                9151 non-null   float64       
 5   release_date              9151 non-null   datetime64[ns]
 6   str_genres                9116 non-null   object        
 7   str_keywords              8380 non-null   object        
 8   genres_keywords_overview  9151 non-null   object        
 9   year                      9151 non-null   int32         
 10  popularity_log            9151 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int32(1), object(7)
memory usage: 750.8+ KB


In [16]:
movies = movies.dropna().reset_index(drop=True)

In [17]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8368 entries, 0 to 8367
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   title                     8368 non-null   object        
 1   genres                    8368 non-null   object        
 2   keywords                  8368 non-null   object        
 3   overview                  8368 non-null   object        
 4   popularity                8368 non-null   float64       
 5   release_date              8368 non-null   datetime64[ns]
 6   str_genres                8368 non-null   object        
 7   str_keywords              8368 non-null   object        
 8   genres_keywords_overview  8368 non-null   object        
 9   year                      8368 non-null   int32         
 10  popularity_log            8368 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int32(1), object(7)
memory usage: 686.6+ KB


## LLM 기반 Contents Based Filtering

In [27]:
from transformers import BertTokenizer,BertModel
import torch

model_name ="bert-base-uncased"



In [28]:
tokenizer = BertTokenizer.from_pretrained(model_name) #영어모델

In [29]:
sentence = "Transformers are amazing!"

tokens = tokenizer.tokenize(sentence)
tokens

['transformers', 'are', 'amazing', '!']

In [30]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
token_ids

[19081, 2024, 6429, 999]

| 항목               | 의미                      | 예시 값                     |
| ---------------- | ----------------------- | ------------------------ |
| `input_ids`      | 문장의 토큰 ID 시퀀스           | `[101, 19081, ..., 102]` |
| `token_type_ids` | 문장 구분 (0: 문장1, 1: 문장2)  | `[0, 0, 0, 0, 0, 0]`     |
| `attention_mask` | 토큰 처리 여부 (1: 처리, 0: 무시) | `[1, 1, 1, 1, 1, 1]`     |

In [31]:
encoded = tokenizer(sentence,return_tensors='pt')
encoded

{'input_ids': tensor([[  101, 19081,  2024,  6429,   999,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [32]:
input_ids = encoded['input_ids'][0]
input_ids

tensor([  101, 19081,  2024,  6429,   999,   102])

In [25]:
decoded_sentence = tokenizer.decode(input_ids)
decoded_sentence

'[CLS] transformers are amazing! [SEP]'

[CLS] (Classification Token) [SEP] (Separator Token)

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

device: cpu


In [ ]:


from sentence_transformers import SentenceTransformer, util
import torch

# 1) 임베딩 모델 선택
# 한국어도 잘 되는 멀티링구얼 모델 (가볍고 실무에서 많이 씀)
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# 2) “게시판 글(문장/문단)” 데이터라고 생각하면 됨
docs = [
    "나는 너 만나 사과를 먹으면서 너에게 잘못했던 문제에 대해서 사과를 했어",
    "나는 사과를 먹으면서 너와 즐겁게 대화를 했어",
    "나는 오늘 데이트를 했는데 사과나무를 봤어",
    "어제 바나나를 먹었어",
    "여름에 바나나보트를 탔어"
]

# 3) 문서 임베딩(한 번만 해두고 저장/DB에 넣어두면 더 좋음)
doc_emb = model.encode(docs, convert_to_tensor=True, normalize_embeddings=True)

def search(query: str, top_k: int = 2):
    # 4) 검색어 임베딩
    q_emb = model.encode(query, convert_to_tensor=True, normalize_embeddings=True)

    # 5) 코사인 유사도(정규화했으니 dot = cosine)
    scores = util.cos_sim(q_emb, doc_emb)[0]  # shape: (len(docs),)

    # 6) Top-K 뽑기
    top = torch.topk(scores, k=min(top_k, len(docs)))
    results = []
    for score, idx in zip(top.values, top.indices):
        results.append((float(score), docs[int(idx)]))
    return results

# ---- 테스트 1: “사과를 먹었어” 검색 ----
query1 = "사과를 먹었어"
print(f"\n[QUERY] {query1}")
for s, text in search(query1):
    print(f"score={s:.4f} | {text}")

# ---- 테스트 2: “즐겁게 대화했어” 검색 ----
query2 = "너와 즐겁게 대화를 했어"
print(f"\n[QUERY] {query2}")
for s, text in search(query2):
    print(f"score={s:.4f} | {text}")

# ---- 테스트 3: “사과(사죄)” 의미로 검색 ----
query3 = "너에게 사과했어(미안하다고)"
print(f"\n[QUERY] {query3}")
for s, text in search(query3):
    print(f"score={s:.4f} | {text}")


In [ ]:
model = BertModel.from_pretrained(model_name)
model.eval() # 모델예측

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
def get_bert_embedding(text):
    inputs = tokenizer(text,return_tensors='pt',truncation=True,padding=True)

    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:,0,:].squeeze().numpy()